In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import math
import time
import os

In [17]:
class Pattention(nn.Module):
    """Pattention Layer.
    d1 = inputs dimension
    d2 = outpuuts dimension
    n = the number of parameters tokens representing the learnable keys and values
    """

    def __init__(
        self,
        d1,
        d2,
        n,
        param_key_init_method,
        param_value_init_method,
        norm_activation_type,
    ):
        super().__init__()

        self.param_token_num = n
        self.param_key_dim = d1
        self.param_value_dim = d2
        self.norm_activation_type = norm_activation_type
        
        self.key_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d1))) # Kp shape of (n, d1) 
        self.value_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d2))) # Vp shape of (n, d2)
        
        param_key_init_method(self.key_param_tokens)
        param_value_init_method(self.value_param_tokens)
    
    def nonlinear_norm_func(self, inputs, normalize_type, dim=-1):
        if normalize_type == 'softmax': 
            # NOTE: softmax = exp_l1_norm
            # outputs = F.softmax(inputs, dim=dim) * inputs.shape[dim]
            nonlinear_outputs = torch.exp(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=1, dim=dim, keepdim=True) * inputs.shape[dim]
            outputs = norm_outputs
        elif normalize_type == 'gelu_l2_norm':
            nonlinear_outputs = F.gelu(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=2, dim=dim, keepdim=True) * math.sqrt(nonlinear_outputs.shape[dim])
            outputs = norm_outputs
        elif normalize_type == 'l2_norm_gelu':
            norm_outputs = inputs / torch.norm(inputs, p=2, dim=dim, keepdim=True) * math.sqrt(inputs.shape[dim])
            nonlinear_outputs = F.gelu(norm_outputs)
            outputs = nonlinear_outputs
        return outputs

    def forward(self, inputs, dropout_p=0.0, attn_mask=None, scale=None):

        query = inputs
        key, value = self.key_param_tokens, self.value_param_tokens        
        L, S = query.size(-2), key.size(-2)
        scale_factor = 1 if scale is None else scale 
        # just for gelu nonlinear, set torch.zeros for softmax
        attn_bias = torch.ones(L, S, dtype=query.dtype, device=query.device)

        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                # just for gelu nonlinear, set -inf for softmax
                attn_bias.masked_fill_(attn_mask.logical_not(), 0)
            else:
                raise NotImplementedError

        attn_weight = query @ key.transpose(-2, -1) * scale_factor
        # just for gelu nonlinear, set attn_weight += attn_bias for softmax
        attn_weight *= attn_bias
        # modified softmax
        attn_weight = self.nonlinear_norm_func(attn_weight, self.norm_activation_type, dim=-1)
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
        output = attn_weight @ value

        return output

In [18]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, attention_dropout=0.1,token_num=10):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads
        #print(hidden_size)
        #print(num_attention_heads)

        assert hidden_size % num_attention_heads == 0, "hidden_size must be divisible by num_attention_heads"

        # Query, Key, and Value projections
        self.query = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.key = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.value = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.out_proj = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )

        self.attention_dropout = nn.Dropout(attention_dropout)
        self.norm_factor = math.sqrt(self.head_dim)

    def forward(self, hidden_states, attention_mask=None):

        # Proceed with attention mechanism
        batch_size, seq_len, _ = hidden_states.size()

        query_layer = self.query(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("qshape: ", query_layer.shape)
        key_layer = self.key(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("kshape: ", key_layer.shape)
        value_layer = self.value(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("vshape: ", value_layer.shape)
        query_layer = query_layer.transpose(1, 2)
        key_layer = key_layer.transpose(1, 2)
        value_layer = value_layer.transpose(1, 2)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores /= self.norm_factor

        if attention_mask is not None:
            attention_scores += attention_mask

        attention_probs = torch.softmax(attention_scores, dim=-1)
        attention_probs = self.attention_dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.transpose(1, 2).contiguous().view(
            batch_size, seq_len, self.hidden_size
        )

        output = self.out_proj(context_layer)
        return output

In [19]:
class TokenformerLayer(nn.Module):
    """A single Tokenformer layer implementing token-token and token-parameter interactions."""

    def __init__(
        self,
        hidden_size,
        vocab_size,
        num_attention_heads,
        max_seq_len,
        attention_dropout=0.1,
        hidden_dropout=0.1,
        token_num=10
    ):
        """
        Args:
            hidden_size (int): The size of the hidden dimension.
            num_attention_heads (int): Number of attention heads for multi-head attention.
            num_param_tokens (int): Number of parameter tokens for the feed-forward Pattention layer.
            attention_dropout (float): Dropout probability for attention weights.
            hidden_dropout (float): Dropout probability for residual connections.
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads

        assert (
            hidden_size % num_attention_heads == 0
        ), "hidden_size must be divisible by num_attention_heads"

        # Layer normalizations
        self.input_layernorm = nn.LayerNorm(hidden_size)
        self.post_attention_layernorm = nn.LayerNorm(hidden_size)

        # Token and positional embeddings
        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_seq_len, hidden_size)

        # Self-attention using Pattention
        self.attention = self.attention = SelfAttention(
            #vocab_size=30522,  # Provide a valid vocab_size if needed
            hidden_size=hidden_size,
            num_attention_heads=num_attention_heads,
            #max_seq_len=max_seq_len,
            attention_dropout=attention_dropout,
            token_num=token_num
        )

        # Feed-forward network using Pattention
        self.mlp = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )

        self.hidden_dropout = hidden_dropout
        self.dropout = nn.Dropout(hidden_dropout)

        # to obtain logits (before softmax) for the vocabulary
        self.lm_head = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, attention_mask=None):
        """
        Forward pass for the Tokenformer layer.
        
        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, seq_len, hidden_size].
            attention_mask (torch.Tensor, optional): Attention mask for self-attention.

        Returns:
            torch.Tensor: Output tensor of the same shape as the input.
        """
        #print(f"Input shape before LayerNorm: {x.shape}")
        # Residual connection and pre-normalization for attention
        # Word embedding
        x = self.token_embedding(x)  # [batch_size, seq_len, hidden_size]
        #print("token emb: ", x.shape)
        # Positional embedding
        seq_len = x.size(1)
        position_ids = torch.arange(seq_len, device=x.device).unsqueeze(0)  # [1, seq_len]
        #print("Position: ", position_ids.shape)
        x += self.position_embedding(position_ids)  # Add positional embeddings
        #print( "emb + pos", x.shape)

        residual = x
        normed_input = self.input_layernorm(x)
        #print("norme: ", normed_input.shape)

        # Self-attention
        attention_output = self.attention(normed_input, attention_mask)
        #print("attention: ", attention_output.shape)
        attention_output = self.dropout(attention_output) + residual

        # Residual connection and pre-normalization for feed-forward
        residual = attention_output
        normed_attention_output = self.post_attention_layernorm(attention_output)

        # Feed-forward network (Pattention)
        mlp_output = self.mlp(normed_attention_output)
        #print("feed: ",  mlp_output.shape)
        output = self.dropout(mlp_output) + residual

        # Linear layer for logits
        logits = self.lm_head(output)

        return logits

In [20]:
# Dataset class remains unchanged
class PokemonDataset(Dataset):
    def __init__(self, file_path, max_seq_len):
        with open(file_path, 'r') as f:
            self.text = f.read().lower().split()
        
        vocab = sorted(set(self.text))
        self.vocab = {word: idx for idx, word in enumerate(vocab)}
        self.vocab_size = len(self.vocab)
        self.max_seq_len = max_seq_len

        # Convert text to token indices
        self.tokens = [self.vocab[word] for word in self.text]
        self.data = [
            self.tokens[i : i + max_seq_len] for i in range(len(self.tokens) - max_seq_len)
        ]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return torch.tensor(x), torch.tensor(y)

In [44]:
def estimate_perplexity(model, dataloader, criterion, device):
    """
    Compute the perplexity of a language model.
    """
    model.eval()  # Set the model to evaluation mode.
    total_loss = 0
    total_tokens = 0

    # Disable gradient computation for evaluation.
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            # Move the batch data to the specified device.
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass through the model.
            outputs = model(x_batch)  # Shape: (batch_size, seq_len, vocab_size)
            outputs = outputs.view(-1, outputs.shape[-1])  # Reshape to (batch_size * seq_len, vocab_size)
            y_batch = y_batch.view(-1)  # Flatten targets to shape: (batch_size * seq_len)

            # Compute the loss for the current batch.
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)  # Multiply by the number of tokens.
            total_tokens += y_batch.size(0)

    # Calculate the average loss per token.
    avg_loss = total_loss / total_tokens
    # Compute the perplexity by exponentiating the average loss.
    perplexity = torch.exp(torch.tensor(avg_loss))
    return avg_loss, perplexity.item()


def train(use_metrics=False):
    # Configurations
    if torch.backends.mps.is_available():
        device = torch.device("mps")  # use M1 GPU
    else:
        device = torch.device("cpu")  # use CPU if no M1 GPU
    hidden_dim=32 
    num_heads=4
    max_seq_len=32
    batch_size=32
    num_epochs=10
    learning_rate=0.001
    token_num=32
    # vocab_size=10000
    #num_layers=2,
    #num_tokens=10,
   

    # Préparer les données
    dataset = PokemonDataset(file_path='training/pokemon.txt', max_seq_len=max_seq_len)
    vocab_size = dataset.vocab_size  # Mettre à jour vocab_size après avoir créé le vocabulaire
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print("vocab_size:", vocab_size)

    # Instancier le modèle
    model = TokenformerLayer(hidden_dim, vocab_size, num_heads, max_seq_len,token_num=token_num)
    model = model.to(device)
    print("model parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
    print("number of training samples:", len(dataset))
    total_tokens = len(dataset) * (max_seq_len - 1) * num_epochs
    print(f"Total training tokens: {total_tokens/1e6:.2f}M")


    # Définir la fonction de perte et l'optimiseur
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print("debut de l'entrainement")
    total_training_time = 0.0  # record the total training time
    overall_start_time = time.time()  # record the overall start time
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        # record the start time of the current epoch
        epoch_train_start = time.time()

        for x_batch, y_batch in dataloader:
            # Move the batch data to the specified device.
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            outputs = model(x_batch)  # Shape: (batch_size, seq_len, vocab_size)

            # Aplatir les logits pour la CrossEntropyLoss
            outputs = outputs.view(-1, vocab_size)  # Shape: (batch_size * seq_len, vocab_size)

            # Aplatir les labels
            y_batch = y_batch.view(-1)  # Shape: (batch_size * seq_len)

            # Calculer la perte
            loss = criterion(outputs, y_batch)

            # Backpropagation et optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # record the end time of the current epoch
        epoch_training_time = time.time() - epoch_train_start
        total_training_time += epoch_training_time

        if use_metrics:
            # record the start time of the evaluation
            eval_start = time.time()
            avg_loss, perplexity = estimate_perplexity(model, dataloader, criterion, device)
            eval_time = time.time() - eval_start
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {epoch_loss / len(dataloader):.4f}, "
                  f"Eval Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}, "
                  f"Train Time: {epoch_training_time:.2f}s, Eval Time: {eval_time:.2f}s")
        else:
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {epoch_loss / len(dataloader):.4f}, "
                  f"Train Time: {epoch_training_time:.2f}s")

    print(f"Total training time (excluding evaluation): {total_training_time:.2f} seconds")

In [45]:
train(use_metrics=True)

vocab_size: 10655
model parameters: 703967
number of training samples: 89205
Total training tokens: 27.65M
debut de l'entrainement


KeyboardInterrupt: 

## Progressive Scalling

In [48]:
class Pattention(nn.Module):
    """Pattention Layer.
    d1 = inputs dimension
    d2 = outpuuts dimension
    n = the number of parameters tokens representing the learnable keys and values
    """

    def __init__(
        self,
        d1,
        d2,
        n,
        param_key_init_method,
        param_value_init_method,
        norm_activation_type,
    ):
        super().__init__()

        self.param_token_num = n
        self.param_key_dim = d1
        self.param_value_dim = d2
        self.norm_activation_type = norm_activation_type
        
        self.key_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d1))) # Kp shape of (n, d1) 
        self.value_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d2))) # Vp shape of (n, d2)
        
        param_key_init_method(self.key_param_tokens)
        param_value_init_method(self.value_param_tokens)

    def add_new_params(self, num_new_tokens):
        device = self.key_param_tokens.device  # Get the current device

        new_key_tokens = nn.Parameter(torch.zeros((num_new_tokens, self.param_key_dim), device=device))
        new_value_tokens = nn.Parameter(torch.zeros((num_new_tokens, self.param_value_dim), device=device))

        # Concatenate with existing parameters
        self.key_param_tokens = nn.Parameter(torch.cat([self.key_param_tokens, new_key_tokens], dim=0))
        self.value_param_tokens = nn.Parameter(torch.cat([self.value_param_tokens, new_value_tokens], dim=0))

        #print(f"Added {num_new_tokens} new tokens. Total tokens now: {self.key_param_tokens.shape[0]}")
    
    def nonlinear_norm_func(self, inputs, normalize_type, dim=-1):
        if normalize_type == 'softmax': 
            # NOTE: softmax = exp_l1_norm
            # outputs = F.softmax(inputs, dim=dim) * inputs.shape[dim]
            nonlinear_outputs = torch.exp(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=1, dim=dim, keepdim=True) * inputs.shape[dim]
            outputs = norm_outputs
        elif normalize_type == 'gelu_l2_norm':
            nonlinear_outputs = F.gelu(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=2, dim=dim, keepdim=True) * math.sqrt(nonlinear_outputs.shape[dim])
            outputs = norm_outputs
        elif normalize_type == 'l2_norm_gelu':
            norm_outputs = inputs / torch.norm(inputs, p=2, dim=dim, keepdim=True) * math.sqrt(inputs.shape[dim])
            nonlinear_outputs = F.gelu(norm_outputs)
            outputs = nonlinear_outputs
        return outputs


    def forward(self, inputs, dropout_p=0.0, attn_mask=None, scale=None):

        query = inputs
        key, value = self.key_param_tokens, self.value_param_tokens        
        L, S = query.size(-2), key.size(-2)
        scale_factor = 1 if scale is None else scale 
        # just for gelu nonlinear, set torch.zeros for softmax
        attn_bias = torch.ones(L, S, dtype=query.dtype, device=query.device)

        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                # just for gelu nonlinear, set -inf for softmax
                attn_bias.masked_fill_(attn_mask.logical_not(), 0)
            else:
                raise NotImplementedError

        attn_weight = query @ key.transpose(-2, -1) * scale_factor
        # just for gelu nonlinear, set attn_weight += attn_bias for softmax
        attn_weight *= attn_bias
        # modified softmax
        attn_weight = self.nonlinear_norm_func(attn_weight, self.norm_activation_type, dim=-1)
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
        output = attn_weight @ value

        return output

In [58]:
def count_parameters(model):
    """Count the total number of trainable parameters in the model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model(model, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")


def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")


from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

def progressive_train_and_scale(
    file_path,
    vocab_size,
    max_seq_len,
    hidden_dim,
    num_heads,
    batch_size,
    num_epochs,
    scaling_steps,
    new_tokens_per_step,
    learning_rate=0.001,
    checkpoint_path='checkpoints/tokenformer_base.pth',
    use_metrics=False,
    test_size=0.1,  # Default 10% of data as test set
):
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'

    # Load dataset
    dataset = PokemonDataset(file_path=file_path, max_seq_len=max_seq_len)
    vocab_size = dataset.vocab_size
    
    # Train-test split
    train_data, val_data = train_test_split(dataset, test_size=test_size, random_state=42)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    # Instantiate the model and move to device
    model = TokenformerLayer(hidden_size=hidden_dim, vocab_size=vocab_size, num_attention_heads=num_heads, max_seq_len=max_seq_len).to(device)

    print("model parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
    print("number of training samples:", len(dataset))
    total_tokens = len(dataset) * (max_seq_len - 1) * num_epochs * (scaling_steps + 1)
    print(f"Total training tokens: {total_tokens/1e6:.2f}M")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel()} parameters")

    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    initial_start_time = time.time()
    print("Starting initial training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch).view(-1, vocab_size)
            y_batch = y_batch.view(-1)

            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if use_metrics:
            avg_epoch_loss = total_loss / len(train_loader)
            eval_loss, perplexity = estimate_perplexity(model, val_loader, criterion, device)
            print(f"Initial Training Epoch {epoch + 1}/{num_epochs}, "
                f"Train Loss: {avg_epoch_loss:.4f}, Eval Loss: {eval_loss:.4f}, Perplexity: {perplexity:.4f}")
        else:
            print(f"Initial Training Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
    
    initial_end_time = time.time()
    print(f"Initial training time: {initial_end_time - initial_start_time:.2f} seconds")
    save_model(model, checkpoint_path)
    print(f"Initial model parameters: {count_parameters(model)}")

    load_model(model, checkpoint_path)
    fine_tuning_start_time = time.time()
    for step in range(scaling_steps):
        for module in model.modules():
            if isinstance(module, Pattention):
                module.add_new_params(new_tokens_per_step)
        print(f"Step {step + 1}: Model parameters after adding new params: {count_parameters(model)}")
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        checkpoint_path_step = f'checkpoints/tokenformer_scaled_step_{step + 1}.pth'
        save_model(model, checkpoint_path_step)
        print(f"Step {step + 1}: Model parameters after scaling: {count_parameters(model)}")
        load_model(model, checkpoint_path_step)

        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            for x_batch, y_batch in train_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch).view(-1, vocab_size)
                y_batch = y_batch.view(-1)

                loss = criterion(outputs, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            
            if use_metrics:
                avg_epoch_loss = total_loss / len(train_loader)
                eval_loss, perplexity = estimate_perplexity(model, val_loader, criterion, device)
                print(f"Step {step + 1}/{scaling_steps}, Fine-tune Epoch {epoch + 1}/{num_epochs}, "
                    f"Train Loss: {avg_epoch_loss:.4f}, Eval Loss: {eval_loss:.4f}, Perplexity: {perplexity:.4f}")
            else:
                print(f"Step {step + 1}/{scaling_steps}, Fine-tune Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
            
    fine_tuning_end_time = time.time()
    print(f"Progressive scaling and fine-tuning time: {fine_tuning_end_time - fine_tuning_start_time:.2f} seconds")
    print(f"Total training time: {fine_tuning_end_time - initial_start_time:.2f} seconds")
    final_checkpoint_path = 'checkpoints/tokenformer_final_scaled.pth'
    save_model(model, final_checkpoint_path)
    print(f"Final scaled model saved to {final_checkpoint_path}")


In [59]:
progressive_train_and_scale(
    file_path='training/pokemon.txt',
    vocab_size=10000,
    max_seq_len=50,
    hidden_dim=32,
    num_heads=4,
    batch_size=32,
    num_epochs=5,
    scaling_steps=3,
    new_tokens_per_step=8,
    use_metrics=True,
)

model parameters: 697503
number of training samples: 89187
Total training tokens: 87.40M
input_layernorm.weight: 32 parameters
input_layernorm.bias: 32 parameters
post_attention_layernorm.weight: 32 parameters
post_attention_layernorm.bias: 32 parameters
token_embedding.weight: 340960 parameters
position_embedding.weight: 1600 parameters
attention.query.key_param_tokens: 320 parameters
attention.query.value_param_tokens: 320 parameters
attention.key.key_param_tokens: 320 parameters
attention.key.value_param_tokens: 320 parameters
attention.value.key_param_tokens: 320 parameters
attention.value.value_param_tokens: 320 parameters
attention.out_proj.key_param_tokens: 320 parameters
attention.out_proj.value_param_tokens: 320 parameters
mlp.key_param_tokens: 320 parameters
mlp.value_param_tokens: 320 parameters
lm_head.weight: 340960 parameters
lm_head.bias: 10655 parameters
Starting initial training...


KeyboardInterrupt: 